# PRONÓSTICO DEL AVALUO DE VIVIENDAS

## CAPITULO I. REDUCCIÓN DE DIMENSIONALIDAD

Buscando lograr una mayor eficiencia en el proceso de pronóstico se realiza una limpieza preliminar de variables buscando incorporar aquellas variables que pueden llegar a explicar de mejor manera el comportamiento del avaluó de viviendas.

### i) Variables que  que por criterio de negocio no se consideran dentro del análisis
- **variables concentradas** variables como objeto, motivo, tipo_credito se encuentran altamente concentradas en una categoria, por ello buscando reducir la complejidad del modelo se eliminan.

- **variables texto** casos como dirección_inmueble y descripciones aportan información valiosa pero en un primer acercamiento no se tendrán en cuenta.


Un mayor detalle de las variables se encuentra en el archivo "Diccionario-variables.xlsx"

In [1]:
import pandas as pd

In [4]:
baseAvaluos = pd.read_csv("train_precios_vivienda.csv", sep = ",", dtype="str")

In [35]:
baseAvaluos['fecha_aprobación'].isna().sum()

np.int64(6774)